In [1]:
lines = open('input1.txt', 'r')
dial = 50
number_of_zeros = 0
number_of_clicks = 0

for line in lines:
	rotation = int(line[1:]) * (-1 if line[0] == 'L' else 1)
	dial %= (100 if rotation > 0 else -100)

	dial += rotation

	number_of_clicks += abs(dial) // 100
	number_of_zeros += 1 if dial % 100 == 0 else 0

print(f'The number of times the dial stops at zero is {number_of_zeros}')
print(f'The number of times the dial touches zero is {number_of_clicks}')

The number of times the dial stops at zero is 992
The number of times the dial touches zero is 6133


In [2]:
ranges_string = open('input2.txt', 'r').read()
ranges = []
for id_range in ranges_string.split(','):
	start, end = id_range.strip().split('-')
	ranges.append((int(start), int(end)))

factorizations = {}
def factorize(number):
	if number in factorizations:
		return factorizations[number]
		
	factors = []
	for i in range(1, number+1):
		if number % i == 0:
			factors.append(i)
	factorizations[number] = factors
	return factors

# Brute force part one
invalids_part_one = set()
invalids = set()

for start, end in ranges:
	for number in range(start, end+1):
		number_string = str(number)
		string_length = len(number_string)
		factors = factorize(string_length)
		for factor in factors:
			length = string_length // factor
			if length == string_length:
				continue
			pieces = [number_string[i:i+length] for i in range(0,string_length,length)]
			is_repeating = all(pieces[i] == pieces[i+1] for i in range(len(pieces)-1))
			if is_repeating:
				invalids.add(number)
				if factor == 2:
					invalids_part_one.add(number)
			
print(f'The sum of the invalid IDs that repeat a string twice is {sum(invalids_part_one)}')
print(f'The sum of the invalid IDs that repeat a string any number of times is {sum(invalids)}')

The sum of the invalid IDs that repeat a string twice is 35367539282
The sum of the invalid IDs that repeat a string any number of times is 45814076230


In [3]:
joltages = open('input3.txt', 'r').read().strip().split('\n')

def find_highest_joltages(joltages, number_of_digits):
	highest = []
	for line in joltages:
		digits = [int(c) for c in line.strip()]
		found_digits = []
		starting_index = 0
		for i in range(number_of_digits-1, -1, -1):
			current_range = digits[starting_index:len(digits)-i]
			current_digit = max(current_range)
			found_digits.append(current_digit)
			starting_index += current_range.index(current_digit) + 1
		result = sum(digit * (10 ** index) for index, digit in enumerate(found_digits[::-1]))
		highest.append(result)
	return highest
	
highest = find_highest_joltages(joltages, 2)		
print(f'The sum of the highest joltages when picking two batteries is {sum(highest)}')

highest = find_highest_joltages(joltages, 12)
print(f'The sum of the highest joltages when picking twelve batteries is {sum(highest)}')

The sum of the highest joltages when picking two batteries is 17158
The sum of the highest joltages when picking twelve batteries is 170449335646486
